In [ ]:
# load chsimpy 
import importlib
import pathlib
import sys

try:
    import chsimpy
except ImportError:
    _parentdir = pathlib.Path("./").resolve().parent
    sys.path.insert(0, str(_parentdir))
    import chsimpy
    #sys.path.remove(str(_parentdir))

In [ ]:
# if code has changed, reload here
importlib.reload(chsimpy)

importlib.reload(chsimpy.parameters)
importlib.reload(chsimpy.solution)
importlib.reload(chsimpy.solver)
importlib.reload(chsimpy.simulator)
importlib.reload(chsimpy.timedata)
importlib.reload(chsimpy.plotview)
importlib.reload(chsimpy.utils)
importlib.reload(chsimpy.mport)

In [ ]:
params = chsimpy.parameters.Parameters()  # initial parameter configuration
params.N = 512  # resolution (NxN image)
params.ntmax = 120  # maximal number of simulation steps
params.threshold = 0.9  # threshold value to determine components A and B in U
params.delt = 1e-11
params.delt_max = 1e-9
params.adaptive_time = True
params.M = 9e-12  # mobility factor
params.full_sim = False  # False -> will stop early when energy falls
params.render_target='gui'  # '+png' renders to PNG, '+yaml' dumps metadata, ... 

# give it to chsimpy controller
simulator = chsimpy.simulator.Simulator(params)

# vars(params)  # show all parameters

In [ ]:
# initial U as needed (you can skip this block)
import numpy as np
rng = np.random.default_rng(seed=2023)
U_0 = 0.875 + (0.01 * (rng.random((params.N, params.N)) - 0.5))
# give it to chsimpy controller
simulator = chsimpy.simulator.Simulator(params, U_0)

In [ ]:
# solve and render it
# simulator.solve()
# simulator.render()
# solution = simulator.solution
solution = simulator.solve()
simulator.render()

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import colors
import matplotlib
import pandas as pd
# matplotlib.artist.getp(fig.patch) # https://matplotlib.org/stable/tutorials/intermediate/artists.html#sphx-glr-tutorials-intermediate-artists-py

In [ ]:
U = solution.U
Ureal = np.real(U)
N = simulator.solver.params.N
th = 0.9  # threshold

# colormap, https://matplotlib.org/2.0.2/users/colormaps.html
cmap = colors.ListedColormap(['orange', 'yellow'])
boundaries = [0.0, th, 1]
norm = colors.BoundaryNorm(boundaries, cmap.N, clip=True)

fig = plt.figure(figsize=(12,5))
# nrow, ncol, index
ax1 = fig.add_subplot(1, 2, 1)

#ax1.imshow(Ureal, cmap=cmap, norm=norm, aspect="equal")
ax1.imshow(Ureal, cmap="plasma", aspect="equal")

ax2 = fig.add_subplot(1, 2, 2) 

#ax.cla()
#ax2.set_xlim(0, N)
#ax2.set_ylim(0.5, 1)

# https://matplotlib.org/2.0.2/users/colormaps.html
ax2.plot(np.arange(0,N), Ureal[int(N / 2)+1,:])

display(fig)
#plt.pause(0.5)
matplotlib.pyplot.close()